# Quickstart Guide

Following the Quickstart Guide. I don't have a personal OpenAI account so I will try to use HuggingfaceHub instead

https://python.langchain.com/en/latest/getting_started/getting_started.html

In [1]:
# load .env
import os
from dotenv import load_dotenv

load_dotenv()

# check HF token and SERPAPI key have been loaded
assert "HUGGINGFACEHUB_API_TOKEN" in os.environ
assert "SERPAPI_API_KEY" in os.environ

## Building a Language Model Application: LLMs

### LLMs: Get predictions from a language model

In [2]:
from langchain.llms import HuggingFaceHub

In [3]:
llm=HuggingFaceHub (repo_id="google/flan-t5-xl", model_kwargs= {"temperature":1e-10})

/Users/eduardo/Library/Caches/pypoetry/virtualenvs/hello-langchain-U8Hlbz7x-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))

sock monkey


### Promp Templates: Manage prompts for LLMs

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [6]:
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


### Chains: Combine LLMs and prompts in multi-step workflows

In [7]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

chain.run("colorful socks")

'sock monkey'

In [8]:
chain.run("witty mugs")

'mugs'

wild one

### Agents: Dynamically Call Chains Based on User Input

In [9]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm=HuggingFaceHub (repo_id="google/flan-t5-xl", model_kwargs= {"temperature":1e-10})

# Let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("When did Real Madrid win La Decima? What is that number multiplied by 3?")



> Entering new AgentExecutor chain...


ValueError: Could not parse LLM output: `Answer: 0`

gave it a few tries but couldn't get it to work. I'll try again when I get an OpenAI account

### Memory: Add State to Chains and Agents

In [10]:
from langchain import ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


'Hi there!'

In [11]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hi there!
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


"I'm doing well too! How are you?"

This is pretty cool :)